In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import cPickle as pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot
import sklearn as sk
import os
from osgeo import ogr
import geopandas as gpd
import operator
import json
from shapely.geometry import Point, shape

In [ ]:
df = pd.read_excel('../NIJ2013_JAN01_DEC31.xlsx')

In [ ]:
df.head()

In [ ]:
col_uniques = []
for i in df.columns:
    col_uniques.append([df[i].unique(), i])

In [ ]:
filter(lambda x: x[1] == 'CASE DESC', col_uniques)

In [ ]:
df[df['final_case_type'] == 'DISTP ']['CASE DESC'].unique()

In [ ]:
for i in df.columns[:-4]:
    df[i] = df[i].str.rstrip()

In [ ]:
excels = []
e = os.listdir('../data/')
for i in e:
    if i.endswith('_Data'):
        files = list(os.walk('../data/' + i))[0]
        excel = filter(lambda x: x.endswith('.xlsx'), files[2])
        excels.append(files[0] + '/' + excel[0])
    else:
        pass

In [ ]:
dataframes = []
for i in excels:
    print 'reading %s' % i
    dataframes.append(pd.read_excel(i))
    print 'read %s' % i
bigDF = pd.concat(dataframes)

In [ ]:
bigDF.to_csv('../data/cleanData/all_crim_data.csv')

In [ ]:
burg = bigDF[bigDF['CATEGORY'] == 'BURGLARY']
burg_counts = burg[['CATEGORY', 'occ_date']].groupby('occ_date').count()

In [ ]:
burg_counts.resample('2Q', 'sum').plot()

In [ ]:
#fig, ax = pyplot.subplots()
#weekly_sample.plot()

In [ ]:
#driver = ogr.GetDriverByName('ESRI Shapefile')
#gridfile = "../data/shapefiles/testfinalclip.shp"
#gridSource = driver.Open(gridfile, 0) # 0 means read-only. 1 means writeable.
#gridLayer = gridSource.GetLayer()
#gridPoints = []
#geoms = []
#for feature in gridLayer:
#    geom = feature.GetGeometryRef()
#    geoms.append(geom)
#    p = geom.GetEnvelope()
#    print p
#    gridPoints.append(p)

In [ ]:
g1 = gpd.GeoDataFrame.from_file("../data/shapefiles/Portland_Police_Districts.shp")
g2 = gpd.GeoDataFrame.from_file("../data/shapefiles/polyGrid.shp")

In [ ]:
g1 = gpd.GeoDataFrame.from_file("../data/shapefiles/Portland_Police_Districts.shp")
g2 = gpd.GeoDataFrame.from_file("../data/shapefiles/polyGrid.shp")data = []
for index, dist in g1.iterrows():
    for index2, grid in g2.iterrows():
        if dist['geometry'].intersects(grid['geometry']):
            data.append({'geometry': dist['geometry'].intersection(grid['geometry']), 'district':dist['DISTRICT'], 
                         'precinct': dist['PRECINCT']})


In [178]:
data[0].keys()

['geometry', 'precinct', 'district']

In [ ]:
geoDF = gpd.GeoDataFrame(data, columns=['district', 'geometry', 'precinct'])
geoDF.to_file('../data/shapefiles/intersections.shp')

In [ ]:
squares = []
for i in range(len(bigDF)):
    if i % 10000 == 0:
        print 'on row %d, %d percent complete' % (i, float(i)/len(bigDF))
    coord = bigDF.iloc[i][['x_coordinate', 'y_coordinate']]
    pt = Point(coord['x_coordinate'], coord['y_coordinate'])
    g = geoDF[geoDF.geometry.contains(pt) == True]['geometry']
    squares.append({'df_index':i, 'square_index': g.index.values[0]})

In [ ]:
#data = []
#for i in range(len(g1)):
#    print i
#    for y in range(len(g3)):
#        if y % 1000 == 0:
#            print y
#        if g1.iloc[i]['geometry'].intersects(g2.iloc[y]['geometry']):
#            data.append({'geometry': g1.iloc[i]['geometry'].intersection(g2.iloc[y]['geometry']), 'district':g1.iloc[i]['DISTRICT'], 
#                       'precinct': g1.iloc[i]['PRECINCT']})

In [ ]:
coord = bigDF.iloc[-1][['x_coordinate', 'y_coordinate']]
pt = Point(coord['x_coordinate'], coord['y_coordinate'])
g = geoDF[geoDF.geometry.contains(pt) == True]['geometry']

In [ ]:
bigDF[926740:926744]

In [ ]:
sq = filter(lambda x: len(x) != 0, squares)

In [ ]:
sq[0].index.values[0]

In [ ]:
ldf = bigDF[10:20]
ldf.iloc[9][['x_coordinate', 'y_coordinate']]

In [ ]:
sq = pickle.load(open('../data/cleanData/testmatched.pkl', 'rb'))

In [ ]:
GEO_DF_PATH = "../data/shapefiles/intersections.shp"

def matchPoints(bigDF):
    geoDF =  gpd.GeoDataFrame.from_file(GEO_DF_PATH)
    squares = []
    for i in range(len(bigDF)):
        if i % 10000 == 0:
            print 'on row %d, %f percent complete' % (i, float(i)/float(len(bigDF)))
        coord = bigDF.iloc[i][['x_coordinate', 'y_coordinate']]
        pt = Point(coord['x_coordinate'], coord['y_coordinate'])
        g = geoDF[geoDF.geometry.contains(pt) == True]['geometry']
        if len(g) > 0:
            squares.append({'df_index':i, 'square_index': g.index.values[0]})
        else:
            squares.append({'df_index':i, 'square_index': None})
        print 'COMPLETED %d rows, %f percent complete' % (i+1, float(i+1)/float(len(bigDF)))
    return squares

In [ ]:
print 'reading pdf'
#pdf = pd.read_csv('../data/cleanData/all_crim_data.csv')
chunk_size = 2
splits = range(0, len(bigDF), chunk_size)
#chunks = map(lambda x: x + chunk_size, splits)
print 'splitting df'
df_list = map(lambda x: bigDF[x:x+chunk_size], splits)
print 'starting matching process'
res = map(lambda x: matchPoints(x), df_list[:2])
print 'completed match'
matched = reduce(operator.add, res)
pklPath = '../data/cleanData/testmatched.pkl'
print 'saving %d matches in %s' % (len(matched), pklPath)
#pickle.dump(matched, open(pklPath, 'wb'))

In [ ]:
res = map(lambda x: matchPoints(x), df_list[:2])
print 'completed match'
matched = reduce(operator.add, res)

In [ ]:
matched = reduce(operator.add, res)

In [ ]:
json.dump(matched, open('testmatched.json', 'wb'))

In [ ]:
json.load(open('testmatched.json'))

In [ ]:
len(bigDF)/13

In [168]:
l = range(5)
reduce(operator.add, l)

10

In [174]:
def useThis(f):
    print f(range(5))
useThis(sum)

10


In [173]:
g1 = gpd.GeoDataFrame.from_file("../data/shapefiles/Portland_Police_Districts.shp")
g2 = gpd.GeoDataFrame.from_file("../data/shapefiles/polyGrid.shp")
data = []
for i in range(len(g1)):
    print i
    for y in range(len(g3)):
        if y % 1000 == 0:
            print y
        if g1.iloc[i]['geometry'].intersects(g2.iloc[y]['geometry']):
            data.append({'geometry': g1.iloc[i]['geometry'].intersection(g2.iloc[y]['geometry']), 
                         'district':g1.iloc[i]['DISTRICT'], 
                       'precinct': g1.iloc[i]['PRECINCT']})

10

In [175]:
len(g1)

60

In [176]:
'a.shp'[:-3]

'a.'